In [1]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [19]:
#Import the relevant libraries
from sklearn import svm
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np


In [17]:
#Load in the demographic dataset
demographicData = pd.read_excel('oasis2_longitudinal_demographics.xlsx')
#Show the head of the data
demographicData.head()

,Subject ID,MRI ID,Group,Visit,MR Delay,M/F,Hand,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
0,OAS2_0001,OAS2_0001_MR1,Nondemented,1,0,M,R,87,14,2.0,27.0,0.0,1986.550000,0.696106,0.883440
1,OAS2_0001,OAS2_0001_MR2,Nondemented,2,457,M,R,88,14,2.0,30.0,0.0,2004.479526,0.681062,0.875539
2,OAS2_0002,OAS2_0002_MR1,Demented,1,0,M,R,75,12,NaN,23.0,0.5,1678.290000,0.736336,1.045710
3,OAS2_0002,OAS2_0002_MR2,Demented,2,560,M,R,76,12,NaN,28.0,0.5,1737.620000,0.713402,1.010000
4,OAS2_0002,OAS2_0002_MR3,Demented,3,1895,M,R,80,12,NaN,22.0,0.5,1697.911134,0.701236,1.033623


In [18]:
#Filter so it's only the third visit
demographicData3 = demographicData[demographicData.Visit ==3]

demographicData3.head()

,Subject ID,MRI ID,Group,Visit,MR Delay,M/F,Hand,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
4,OAS2_0002,OAS2_0002_MR3,Demented,3,1895,M,R,80,12,NaN,22.0,0.5,1697.911134,0.701236,1.033623
9,OAS2_0005,OAS2_0005_MR3,Nondemented,3,1603,M,R,85,12,4.0,30.0,0.0,1699.269072,0.705081,1.032797
11,OAS2_0007,OAS2_0007_MR3,Demented,3,518,M,R,73,16,NaN,27.0,1.0,1364.500000,0.727032,1.286180
21,OAS2_0012,OAS2_0012_MR3,Nondemented,3,1598,F,R,83,16,2.0,29.0,0.0,1322.611763,0.718238,1.326920
24,OAS2_0013,OAS2_0013_MR3,Nondemented,3,1456,F,R,85,12,4.0,29.0,0.0,1224.664107,0.709550,1.433046


In [13]:
#Establish the variables we are fitting torwards
y_MMSE = demographicData3['MMSE']
y_CDR = demographicData3['CDR']

#This function splits the x data and y data into training and testing cohorts (50% go to test cohort)
xTrain, xTest, MMSE_Train, MMSE_Test = train_test_split(x, y_MMSE, test_size=0.5)
xTrain, xTest, CDR_Train, CDR_Test = train_test_split(x, y_CDR, test_size=0.5)


NameError: name 'data' is not defined

In [ ]:
#Functions for training the SVM and the RF

def runSVM(xTraining, yTraining, xTesting, yTesting, kernelType, tuneHyperparameters):
    svRegress = svm.SVR(kernel=kernelTye)
    if tuneHyperparameters:
        
    svRegress.fit(xTraining,yTraining)
    svRegress_predict = svRegress.pred(xTesting)
    svRegress_acc = metrics.accuracy_score(yTesting, svRegress_predict)
    return svRegress, svRegress_predict, svRegress_acc

def runRF(xTraining, yTraining, xTesting, yTesting, tuneHyperparameters):
    randForest = MMSE_Forest = RandomForestRegressor()
    randForest.fit(xTraining,yTraining)
    randForest_predict = randForest.pred(xTesting)
    randForest_acc = metrics.accuracy_score(yTesting, randForest_predict)
    return randForest, randForest_predict, randForest_acc

In [ ]:
#Create the SVC to predict MMSE
MMSE_SVR, MMSE_SVR_predict, MMSE_SVR_acc = runSVM(xTrain, MMSE_Train, xTest, MMSE_Test, 'linear', False)
print("Support Vector Regression MMSE Accuracy: ", MMRE_SVR_acc)

In [ ]:
#Create the SVC to predict CDR
CDR_SVR, CDR_SVR_predict, CDR_SVR_acc = runSVM(xTrain, CDR_Train, xTest, CDR_Test, 'linear', False)
print("Support Vector Regression MMSE Accuracy: ", CDR_SVR_acc)

In [ ]:
#Create the RF to predict MMSE
MMSE_RF, MMSE_RF_predict, MMSE_RF_acc = runRF(xTrain, MMSE_Train, xTest, MMSE_Test, False)
print("Random Forest MMSE Accuracy: ", MMRE_RF_acc)

In [ ]:
#Create the RF to predict CDR
CDR_RF, CDR_RF_predict, CDR_RF_acc = runRF(xTrain, CDR_Train, xTest, CDR_Test, False)
print("Random Forest CDR Accuracy: ", MMRE_RF_acc)